In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('test') \
                    .getOrCreate()

24/04/23 17:04:25 WARN Utils: Your hostname, codespaces-201091 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/04/23 17:04:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 17:04:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
static = spark.read.json('../data/activity-data/')
dataSchema = static.schema

In [49]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json('../data/activity-data/')

In [5]:
streaming.isStreaming

True

In [7]:
streaming.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [ ]:
streaming.writeStream.trigger(once=True).format("console").start()

# Всякое

In [15]:
a = f.lit('2024-02-19 12:47:36') < (f.lit('2024-02-09 13:00:00') + f.expr('INTERVAL 23 HOURS'))
b = f.expr('from_utc_timestamp("2024-02-19 12:47:36", "Asia/Jerusalem")')

In [16]:
static.withColumn('a', a).withColumn('b', b).show(1)

+-------------+-------------------+--------+-----+------+----+-----+-----------+------------+------------+-----+-------------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|          x|           y|           z|    a|                  b|
+-------------+-------------------+--------+-----+------+----+-----+-----------+------------+------------+-----+-------------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand|3.356934E-4|-5.645752E-4|-0.018814087|false|2024-02-19 14:47:36|
+-------------+-------------------+--------+-----+------+----+-----+-----------+------------+------------+-----+-------------------+
only showing top 1 row



In [5]:
activityCounts = streaming.groupBy("gt").count()

In [6]:
activityQuery = activityCounts.writeStream.queryName("activity_counts").format("memory").outputMode("complete").start()

24/04/04 09:13:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a39174c3-93bc-4d83-ab54-a91e47e3a2a4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/04 09:13:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
from time import sleep

for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

In [ ]:
from pyspark.sql.functions import expr

simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
    .where("stairs")\
    .where("gt is not null")\
    .select("gt", "model", "arrival_time", "creation_time")\
    .writeStream\
    .queryName("simple_transform")\
    .format("memory")\
    .outputMode("append")\
    .start()

In [ ]:
for x in range(5):
    spark.sql("SELECT * FROM simple_transform").show()
    sleep(1)

# Разовый прогон для дебага

In [7]:
distDevice = streaming.select('Device').distinct().writeStream.trigger(once=True).queryName('dist_device').format('memory').outputMode("append").start()
distDevice.awaitTermination()

24/04/05 11:49:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-23d839c7-d38d-4b69-946d-69d9b200a73d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 11:49:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/05 11:49:02 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/activity-data] is ignored when Trigger.Once is used.


In [9]:
spark.sql("SELECT * FROM dist_device").show()

+--------+
|  Device|
+--------+
|nexus4_1|
|nexus4_2|
+--------+



In [9]:
streaming.select('Device').distinct().groupBy('Device').count().writeStream.trigger(once=True).format("console").outputMode("complete").start()

24/04/05 11:15:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7b5ac24f-a735-4d67-add5-a40353c0d833. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 11:15:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/05 11:15:57 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/activity-data] is ignored when Trigger.Once is used.


# Пробую

In [50]:
streaming.columns

['Arrival_Time',
 'Creation_Time',
 'Device',
 'Index',
 'Model',
 'User',
 'gt',
 'x',
 'y',
 'z']

In [51]:
filtered = streaming.select('Device', 'gt').distinct()
a = filtered.writeStream.trigger(once=True).queryName('a').format('memory').outputMode("append").start()
a.awaitTermination()

24/04/05 14:33:46 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-662d21dc-bb4e-428a-bb72-bba8b6cfed97. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 14:33:46 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/05 14:33:47 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/activity-data] is ignored when Trigger.Once is used.


In [52]:
spark.sql('select * from a').show()

+--------+----------+
|  Device|        gt|
+--------+----------+
|nexus4_1|      bike|
|nexus4_1|       sit|
|nexus4_2|       sit|
|nexus4_2|  stairsup|
|nexus4_1|stairsdown|
|nexus4_2|      null|
|nexus4_1|  stairsup|
|nexus4_2|      walk|
|nexus4_1|      null|
|nexus4_1|     stand|
|nexus4_1|      walk|
|nexus4_2|      bike|
|nexus4_2|stairsdown|
|nexus4_2|     stand|
+--------+----------+



In [54]:
filtered2 = filtered.filter('Device = "nexus4_2"')
b = filtered2.writeStream.trigger(once=True).queryName('b').format('memory').outputMode("append").start()
b.awaitTermination()

24/04/05 14:35:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6102e6b9-4c75-40eb-82ea-76a9efcd3270. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 14:35:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/05 14:35:49 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/activity-data] is ignored when Trigger.Once is used.


In [25]:
spark.sql('select * from b').show()

+--------+----------+
|  Device|        gt|
+--------+----------+
|nexus4_2|       sit|
|nexus4_2|  stairsup|
|nexus4_2|      null|
|nexus4_2|      walk|
|nexus4_2|      bike|
|nexus4_2|stairsdown|
|nexus4_2|     stand|
+--------+----------+



# Flight Data

In [3]:
static = spark.read.json('../data/flight-data/json/2010-summary.json')
dataSchema = static.schema

In [11]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json('../data/flight-data/json/')

In [5]:
streaming.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



A.number where origin is "Ireland".

In [12]:
a = streaming.filter("ORIGIN_COUNTRY_NAME = 'Ireland'")

In [13]:
a.writeStream.trigger(once=True).format("console").start()

24/04/05 12:33:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-00020e41-cda2-4497-aef5-77ec123c16bc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 12:33:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/05 12:33:43 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/flight-data/json] is ignored when Trigger.Once is used.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Ireland|  344|
|    United States|            Ireland|  264|
|    United States|            Ireland|  268|
|    United States|            Ireland|  266|
|    United States|            Ireland|  252|
|    United States|            Ireland|  291|
+-----------------+-------------------+-----+



All counts

In [14]:
a1 = a.groupBy('ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME').sum('count')

In [16]:
a1.writeStream.trigger(once=True).format("console").outputMode("complete").start()

24/04/05 12:37:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-892352dc-226f-4267-8d26-5ac14d3d82b2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 12:37:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/05 12:37:08 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/flight-data/json] is ignored when Trigger.Once is used.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-----------------+----------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|sum(count)|
+-------------------+-----------------+----------+
|            Ireland|    United States|      1685|
+-------------------+-----------------+----------+



C. Top Origin on cnt (without USA)

In [18]:
from pyspark.sql.functions import desc, asc

c = streaming.filter("ORIGIN_COUNTRY_NAME <> 'United States'").select("ORIGIN_COUNTRY_NAME", "count") \
.groupBy("ORIGIN_COUNTRY_NAME").sum("count").withColumnRenamed("sum(count)", "destination_total") \
.sort(desc("destination_total"))

c.writeStream.trigger(once=True).format("console").outputMode("complete").start()

24/04/05 13:08:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c5e4baf1-34db-4b0c-a32a-afc7009953e4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 13:08:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/05 13:08:48 WARN MicroBatchExecution: The read limit MaxFiles: 1 for FileStreamSource[file:/workspaces/Spark-The-Definitive-Guide/data/flight-data/json] is ignored when Trigger.Once is used.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|destination_total|
+-------------------+-----------------+
|             Canada|            49695|
|             Mexico|            38225|
|     United Kingdom|            10358|
|              Japan|             8643|
|            Germany|             8380|
| Dominican Republic|             7194|
|        The Bahamas|             5775|
|             France|             5290|
|           Colombia|             4981|
|        South Korea|             4253|
|            Jamaica|             4087|
|              China|             4021|
|        Netherlands|             3779|
|             Brazil|             3427|
|         Costa Rica|             3299|
|        El Salvador|             2832|
|              Spain|             2550|
|             Panama|             2480|
|              Italy|             2413|
|           Honduras|  

# Booking data covid

In [3]:
data = [{
    "city": "Amsterdam",
    "country": "Netherlands",
    "timestamp": "2020-01-01T00:00:00",
    "new_cases": 1200,
    "reporter": "AMS01"
}, {
    "city": "Amsterdam",
    "country": "Netherlands",
    "timestamp": "2020-01-01T10:00:00",
    "new_cases": 1300,
    "reporter": "AMS02"
}, {
    "city": "Amsterdam",
    "country": "Netherlands",
    "timestamp": "2020-01-01T12:00:00",
    "new_cases": 1400,
    "reporter": "AMS01"
}, {
    "city": "Utrecht",
    "country": "Netherlands",
    "timestamp": "2020-01-01T00:00:00",
    "new_cases": 100,
    "reporter": "UTR01"
}, {
    "city": "Maastricht",
    "country": "Netherlands",
    "timestamp": "2020-01-04T00:00:00",
    "new_cases": 35,
    "reporter": "MAAS01"
}, {
    "city": "Maastricht",
    "country": "Netherlands",
    "timestamp": "2020-01-05T00:00:00",
    "new_cases": 35,
    "reporter": "MAAS01"
}, {
    "city": "Paris",
    "country": "France",
    "timestamp": "2020-01-05T00:00:00",
    "new_cases": 312,
    "reporter": "PAR01"
}]

In [4]:
import pandas as pd  
  
df = pd.DataFrame.from_records(data,index=['1', '2', '3', '4', '5', '6', '7']) 
print(df)

         city      country            timestamp  new_cases reporter
1   Amsterdam  Netherlands  2020-01-01T00:00:00       1200    AMS01
2   Amsterdam  Netherlands  2020-01-01T10:00:00       1300    AMS02
3   Amsterdam  Netherlands  2020-01-01T12:00:00       1400    AMS01
4     Utrecht  Netherlands  2020-01-01T00:00:00        100    UTR01
5  Maastricht  Netherlands  2020-01-04T00:00:00         35   MAAS01
6  Maastricht  Netherlands  2020-01-05T00:00:00         35   MAAS01
7       Paris       France  2020-01-05T00:00:00        312    PAR01


In [5]:
sparkDF=spark.createDataFrame(df)

You may only use the following given operators of a hypothetical distributed stream processing framework:

filter(filterFunc: T = > Boolean): Stream[T]

Return a new stream of type T containing only the elements that satisfy a predicate(a function of type T that returns either True or False).

map[U](mapFunc: (T) = > U): Stream[U]

Return a new stream of type U by applying a function to all elements of the input stream of type T.

reduceByKey(reduceFunc: (V, V) = > V): Stream[(K, V)]

## Find the list of Dutch cities present in the stream.

In [8]:
sparkDF.filter('country = "Netherlands"').rdd \
    .map(lambda x: (x["city"], x)) \
    .reduceByKey(lambda x, y: x) \
    .map(lambda x: x[0]) \
    .collect()

['Amsterdam', 'Utrecht', 'Maastricht']

## A.The daily number of cases per reporter.

In [62]:
sparkDF.select('new_cases', 'reporter', F.date_trunc('day', 'timestamp').alias('day')).groupBy('day', 'reporter').sum('new_cases').withColumnRenamed("sum(new_cases)", "new_cases_total").show()

+-------------------+--------+---------------+
|                day|reporter|new_cases_total|
+-------------------+--------+---------------+
|2020-01-01 00:00:00|   AMS01|           2600|
|2020-01-04 00:00:00|  MAAS01|             35|
|2020-01-01 00:00:00|   UTR01|            100|
|2020-01-01 00:00:00|   AMS02|           1300|
|2020-01-05 00:00:00|   PAR01|            312|
|2020-01-05 00:00:00|  MAAS01|             35|
+-------------------+--------+---------------+



In [63]:
sparkDF.rdd \
    .map(lambda x: ((x['reporter'], x['timestamp'][:10]), x['new_cases'])) \
    .reduceByKey(lambda x, y: x + y) \
    .collect()

[(('AMS01', '2020-01-01'), 2600),
 (('AMS02', '2020-01-01'), 1300),
 (('UTR01', '2020-01-01'), 100),
 (('MAAS01', '2020-01-04'), 35),
 (('MAAS01', '2020-01-05'), 35),
 (('PAR01', '2020-01-05'), 312)]

## B.Total number of cases per city and date.

In [64]:
sparkDF.rdd \
    .map(lambda x: ((x['city'], x['timestamp'][:10]), (x['country'], x['new_cases']))) \
    .reduceByKey(lambda x, y: (x[0], x[1] + y[1])) \
    .collect()

[(('Amsterdam', '2020-01-01'), ('Netherlands', 3900)),
 (('Utrecht', '2020-01-01'), ('Netherlands', 100)),
 (('Maastricht', '2020-01-04'), ('Netherlands', 35)),
 (('Maastricht', '2020-01-05'), ('Netherlands', 35)),
 (('Paris', '2020-01-05'), ('France', 312))]

## C. For each day, return the top most infected city per country.

In [65]:
sparkDF.rdd \
    .map(lambda x: ((x['country'], x['timestamp'][:10], x['city']), x['new_cases'])) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: ((x[0][0], x[0][1]), (x[0][2], x[1]))) \
    .reduceByKey(lambda x, y: x if x[1] > y[1] else y) \
    .collect()

[(('Netherlands', '2020-01-01'), ('Amsterdam', 3900)),
 (('Netherlands', '2020-01-04'), ('Maastricht', 35)),
 (('Netherlands', '2020-01-05'), ('Maastricht', 35)),
 (('France', '2020-01-05'), ('Paris', 312))]

## D. For each month, return the top 5 infected cities per country.

In [76]:
from datetime import datetime
# 2020-01-01T00:00:00

sparkDF.rdd \
    .map(lambda x: ((x['country'], datetime.strptime(x['timestamp'], '%Y-%m-%dT%H:%M:%S').replace(day=1).strftime("%Y-%m-%d"), x['city']), x['new_cases'])) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: ((x[0][0], x[0][1]), [(x[0][2], x[1])])) \
    .reduceByKey(lambda x, y: x + y) \
    .mapValues(lambda x: sorted(x, key = lambda x: x[1], reverse=True)[:2]) \
    .collect()

[(('Netherlands', '2020-01-01'), [('Amsterdam', 3900), ('Utrecht', 100)]),
 (('France', '2020-01-01'), [('Paris', 312)])]

## agg

In [8]:
sparkDF.agg(
    F.max(F.col("new_cases")).alias("max_new_cases"),
    F.min(F.col("new_cases")).alias("min_new_cases")
).show(5, False)

+-------------+-------------+
|max_new_cases|min_new_cases|
+-------------+-------------+
|1400         |100          |
+-------------+-------------+



# Booking data hotel

In [3]:
data = [{
    "hotel_id": 23,
    "hotel_area_code": "IT",
    "unique_user_id": "user_1",
    "is_user_logged_in": True
}, {
    "hotel_id": 23,
    "hotel_area_code": "IT",
    "unique_user_id": "user_2",
    "is_user_logged_in": True
}, {
    "hotel_id": 23,
    "hotel_area_code": "IT",
    "unique_user_id": "user_1",
    "is_user_logged_in": True
}, {
    "hotel_id": 42,
    "hotel_area_code": "IT",
    "unique_user_id": "user_3",
    "is_user_logged_in": True
}, {
    "hotel_id": 42,
    "hotel_area_code": "IT",
    "unique_user_id": "user_4",
    "is_user_logged_in": False
}, {
    "hotel_id": 88,
    "hotel_area_code": "FR",
    "unique_user_id": "user_1",
    "is_user_logged_in": True
}]

In [4]:
import pandas as pd  
  
df = pd.DataFrame.from_records(data) 
print(df)

   hotel_id hotel_area_code unique_user_id  is_user_logged_in
0        23              IT         user_1               True
1        23              IT         user_2               True
2        23              IT         user_1               True
3        42              IT         user_3               True
4        42              IT         user_4              False
5        88              FR         user_1               True


In [5]:
sparkDF=spark.createDataFrame(df)

## For each area code the hotel with the higher number of views by logged in users

In [6]:
sparkDF.filter('is_user_logged_in == True').rdd \
    .map(lambda x: ((x['hotel_area_code'], x['hotel_id']), (1))) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0][0], (x[0][1], x[1]))) \
    .reduceByKey(lambda x, y: x if x[1] > y[1] else y) \
    .collect()

[('IT', (23, 3)), ('FR', (88, 1))]

## * Distinct area codes

In [24]:
sparkDF.rdd \
    .map(lambda x: (x['hotel_area_code'], '')) \
    .reduceByKey(lambda x, y: x) \
    .collect()

[('IT', ''), ('FR', '')]

# Booking tasks from medium/leetcode

## Map Reduce

In [8]:
data = [
    ("a-b", "data1", 1),
    ("a-c", "data2", 1),
    ("a-b", "data3", 1)
    ]

import pandas as pd  
  
df = pd.DataFrame.from_records(data) 
print(df)

sparkDF=spark.createDataFrame(df)

     0      1  2
0  a-b  data1  1
1  a-c  data2  1
2  a-b  data3  1


In [13]:
#  ("a-b", ["data1", "data3"], 2) ("a-c", ["data2"], 1)
sparkDF.rdd \
    .map(lambda x: (x[0],([x[1]], x[2]))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .collect()

[('a-b', (['data1', 'data3'], 2)), ('a-c', (['data2'], 1))]

## Table Build Order
As part of a job submission to a server, that is responsible for orchestrating the creation of tables for consumption by reports, a piece of metadata is submitted that declares the dependencies of the various tables for the given job.It is the responsibility of the server to make sure that a correct build order is chosen and executed.

In [1]:
dependencies = {
    "TableC": ["TableD"],
    "TableA": ["TableB", "TableC"],
    "TableE": ["TableB"],
    "TableD": ["TableB"]
}

In [18]:
ans = []
tables = set()
# get set of all differen tables
for k, v in dependencies.items():
    tables.add(k)
    for t in v:
        tables.add(t)

# add tables withot dependencies to the dependencies dict
for table in tables:
    if table not in dependencies.keys():
        dependencies[table] = []

# find root
non_root = []
for k, v in dependencies.items():
    non_root += list(v)
roots = [k for k, v in dependencies.items() if k not in non_root]

# dfs
def dfs(table):
    for dep in dependencies[table]:
        dfs(dep)
    if table not in ans:
        ans.append(table)

# start
for root in roots:
    dfs(root)  
print(ans)
    
    

['TableB', 'TableD', 'TableC', 'TableA', 'TableE']


## Merge hotels
Merge HotelRoom object when highPrice + 1 = lowPrice of other object
 - Ex.1  [1,2], [3,4] ==> [1,4]
 - Ex.2  [1,5],[6,9] ==> [1,9]
 - Ex.3 [1,5], [14, 17], [6,9], [10,13] ==> [1,17]
 - Ex.3 [1,5], [14, 17], [6,9], [10,13], [4,7], [8,12] ==> [1,17], [4,12]

In [8]:
arr_obj = [[1,2], [3,4]]
arr_obj.sort(key=lambda x: x[0])
print(arr_obj)

[[1, 2], [3, 4]]


In [9]:
ans = []
for cur in arr_obj:
    print(cur)
    merged = False
    for i in range(len(ans)):
        if ans[i][1] + 1 == cur[0]:
            ans[i][1] = cur[1]
            merged = True
            break
        if ans[i][0] == cur[1] + 1:
            ans[i][0] = cur[0]
            merged = True
            break
    if not merged:
        ans.append(cur)

    print('ans', ans)

print(ans)

[1, 2]
ans [[1, 2]]
[3, 4]
ans [[1, 4]]
[[1, 4]]


## Backspaceing

In [10]:
s = 'abc#def##'

In [12]:
sa = []
for c in s:
    if c != '#':
        sa.append(c)
    else:
        sa.pop()
print(''.join(sa))

abd


## Check-in/out

In [28]:
d = {

	176 : 	[
				{
					"price" : 120,
					"features" : ["breakfast", "refundable"],
					"availbility" : 5
				}
			],
	177 : 	[
				{
					"price" : 130,
					"features" : ["breakfast"],
					"availbility" : 1
				},
				{
					"price" : 140,
					"features" : ["breakfast", "refundable", "wifi"],
					"availbility" : 3
				}
			],
	178 : 	[
				{
					"price" : 130,
					"features" : ["breakfast"],
					"availbility" : 2
				},
				{
					"price" : 140,
					"features" : ["breakfast", "refundable", "wifi"],
					"availbility" : 10
				}
			]
}

In [29]:
input = {
	"checkin" : 176,
	"checkout" : 178,
	"features" : ["breakfast"],
	"rooms"    : 1
}

Expected output

[
	{
		"price" : 250,
		"features" : ["breakfast"],
		"availbility" : 1
	},
	{
		"price" : 260,
		"features" : ["breakfast", "refundable"],
		"availbility" : 3
	}
]

In [30]:
ans = []

# for i in range(input['checkin'], input['checkout'] + 1):
def dfs(i, cur):
    if i == input['checkout']:
        ans.append(cur)
        return
    for option in d[i]:
        if all(feature in option['features'] for feature in input['features']) and option['availbility'] >= input['rooms']:
            if cur:
                new_cur = {}
                new_cur['price'] = cur['price'] + option['price']
                new_cur['features'] = list(set(cur['features']) & set(option['features']))
                new_cur['availbility'] = min(cur['availbility'], option['availbility'])
            else:
                new_cur = {}
                new_cur['price'] = option['price']
                new_cur['features'] = set(option['features'])
                new_cur['availbility'] = option['availbility']
            dfs(i+1, new_cur)

dfs(input['checkin'], {})
    
    

In [31]:
print(ans)

[{'price': 250, 'features': ['breakfast'], 'availbility': 1}, {'price': 260, 'features': ['breakfast', 'refundable'], 'availbility': 3}]


## Given a stream of rows
in the following format, find the cities which has 3 or more hotels with same name
hotelId, hotel Name, City Name

In [3]:
data = [{
    "hotel_id": 1,
    "hotel_name": "A",
    "city": "Amsterdam",
}, {
    "hotel_id": 2,
    "hotel_name": "B",
    "city": "Amsterdam",
}, {
    "hotel_id": 3,
    "hotel_name": "C",
    "city": "Amsterdam",
}, {
    "hotel_id": 4,
    "hotel_name": "A",
    "city": "Amsterdam",
}, {
    "hotel_id": 4,
    "hotel_name": "A",
    "city": "Amsterdam",
}, {
    "hotel_id": 5,
    "hotel_name": "A",
    "city": "Amsterdam",
}, {
    "hotel_id": 6,
    "hotel_name": "A",
    "city": "Paris",
}, {
    "hotel_id": 7,
    "hotel_name": "A",
    "city": "Paris",
}]

In [4]:
import pandas as pd  
  
df = pd.DataFrame.from_records(data) 
print(df)

   hotel_id hotel_name       city
0         1          A  Amsterdam
1         2          B  Amsterdam
2         3          C  Amsterdam
3         4          A  Amsterdam
4         4          A  Amsterdam
5         5          A  Amsterdam
6         6          A      Paris
7         7          A      Paris


In [5]:
sparkDF=spark.createDataFrame(df)

In [9]:
sparkDF.rdd \
    .map(lambda x: ((x['city'], x['hotel_name']), 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .filter() \
    .collect()

[(('Amsterdam', 'A'), 3),
 (('Amsterdam', 'B'), 1),
 (('Amsterdam', 'C'), 1),
 (('Paris', 'A'), 1)]

In [7]:
sparkDF.groupBy(['city', 'hotel_name']) \
    .agg(f.countDistinct('hotel_id').alias('cnt')) \
    .filter('cnt >= 2') \
    .show()

+---------+----------+---+
|     city|hotel_name|cnt|
+---------+----------+---+
|    Paris|         A|  2|
|Amsterdam|         A|  3|
+---------+----------+---+



# RollUp

In [12]:
df = spark.createDataFrame([(2, "Alice"), (7, "Alice"), (5, "Bon")], schema=["age", "name"])
df.rollup("name", "age").agg(f.col("name"), f.col("age")).orderBy("name", "age").show()

+-----+----+-----+----+
| name| age| name| age|
+-----+----+-----+----+
| NULL|NULL| NULL|NULL|
|Alice|NULL|Alice|NULL|
|Alice|   2|Alice|   2|
|Alice|   7|Alice|   7|
|  Bon|NULL|  Bon|NULL|
|  Bon|   5|  Bon|   5|
+-----+----+-----+----+

